In [4]:
import os
import mysql.connector
import pandas as pd
import logging
import time

# ---------------- LOGGING ----------------
logging.basicConfig(
    filename="import_logs.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ---------------- DB CONFIG ----------------
config = {
    'user': 'root',
    'password': 'PAQ2026@',
    'host': 'localhost',
    'database': 'inventory',
    'port': 3306
}

# ---------------- CONNECT TO MYSQL ----------------
conn = mysql.connector.connect(
    user=config['user'],
    password=config['password'],
    host=config['host'],
    database=config['database'],
    allow_local_infile=True   # ✅ enable LOCAL INFILE
)
cursor = conn.cursor()

folder_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads'

# ---------------- PROCESS FILES ----------------
overall_start = time.time()

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        table_name = os.path.splitext(filename)[0]
        file_path = os.path.join(folder_path, filename)

        start_time = time.time()
        print(f"Processing {filename}...")

        try:
            # Read only header to get column names
            df_head = pd.read_csv(file_path, nrows=0)
            columns = df_head.columns.tolist()

            # Generate CREATE TABLE query (all columns VARCHAR initially)
            cols_query = ", ".join([f"`{col}` VARCHAR(255)" for col in columns])
            create_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({cols_query});"
            cursor.execute(create_query)
            conn.commit()

            # Use LOAD DATA LOCAL INFILE to bulk load CSV
            load_query = f"""
            LOAD DATA LOCAL INFILE '{file_path.replace("\\","/")}'
            INTO TABLE `{table_name}`
            FIELDS TERMINATED BY ',' 
            ENCLOSED BY '"'
            LINES TERMINATED BY '\\n'
            IGNORE 1 ROWS;
            """
            cursor.execute(load_query)
            conn.commit()

            elapsed = time.time() - start_time
            logging.info(f"Completed {filename} in {elapsed:.2f} seconds")
            print(f"Completed {filename} in {elapsed:.2f} seconds")

        except Exception as e:
            logging.error(f"Error processing {filename}: {e}")
            print(f"Error processing {filename}: {e}")

cursor.close()
conn.close()

overall_elapsed = time.time() - overall_start
print(f"All files imported successfully in {overall_elapsed:.2f} seconds")


Processing begin_inventory.csv...
Completed begin_inventory.csv in 2.58 seconds
Processing end_inventory.csv...
Completed end_inventory.csv in 2.49 seconds
Processing purchases.csv...
Completed purchases.csv in 31.57 seconds
Processing purchase_prices.csv...
Completed purchase_prices.csv in 0.26 seconds
Processing sales.csv...
Completed sales.csv in 129.37 seconds
Processing vendor_invoice.csv...
Completed vendor_invoice.csv in 0.28 seconds
All files imported successfully in 166.55 seconds
